***Changes To Do:***


1. Change the embedding layer to Bertweet
2. Dataset need to get data from CSV and getitem returns ids
3. During the init of the Dataset we need to find the longest sequence (Including all the info), and add the <pad> token in the end of each sequence before I create the ids.



**Importing libraries**

In [1]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

**Installing missing modules**

In [2]:
!pip install transformers
!pip3 install emoji

     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 47.2 MB/s 
     |████████████████████████████████| 3.3 MB 65.2 MB/s 
     |████████████████████████████████| 636 kB 88.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 184 kB 4.1 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=2ccc98fb8176f4cdfdcc64a087025a59f69de906bd5b8c5ea3c18f6de53497f7
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji


In [3]:
from transformers import AutoModel, AutoTokenizer

**Pulling Data from GDrive**

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import sys
from pathlib import Path

base = Path('/content/gdrive/MyDrive/Deep Learning Project')
sys.path.append(str(base))

zip_path = base/'Dataset.rar'
print(zip_path)
!cp "{zip_path}" .
!unrar x Dataset.rar
!rm Dataset.rar

/content/gdrive/MyDrive/Deep Learning Project/Dataset.rar

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Dataset.rar

Extracting  Hate.csv                                                       5%  OK 
Extracting  Love.csv                                                      10%  OK 
Extracting  Sadness.csv                                                   15%  OK 
Extracting  Surprised.csv                                                 20%  OK 
Extracting  test.csv                                                      30%  OK 
Extracting  train.csv                                                     75%  OK 
Extracting  valid.csv                                                     84%  OK 
Extracting  Anger.csv                                                     90%  OK 
Extracting  Depression.csv                                                95%  OK 
Extracting  Happ

In [8]:
class TweetsDatset(torch.utils.data.Dataset):
  def __init__(self, df, shuffle = False):
    self.df = df
    if shuffle:
      self.df = self.df.sample(frac=1)

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    row_data = self.df.iloc[index]
    raw_tweet = row_data['tweet']
    label = categories_index[row_data['label']]
    tokenized_tweet = bertweet_tokenizer.encode(raw_tweet)
    padding_count = max_word_count - len(tokenized_tweet) + 2
    tokens = bertweet_tokenizer.convert_ids_to_tokens(tokenized_tweet)
    for i in range(padding_count):
      tokens.insert(-2, '<pad>')

    data = torch.tensor(bertweet_tokenizer.convert_tokens_to_ids(tokens))
    print(f' data size:{data.size()}, tokens: {tokens}')

    return data, label

In [9]:
class LSTM_CNN(nn.Module):
  def __init__(self, 
               bert_model, 
               lstm_input_size, 
               lstm_hidden_size,
               lstm_num_layers,
               kernels,
               output_size):
    super().__init__()
    self.bert_embedding = bert_model
    self.lstm = nn.LSTM(input_size= lstm_input_size, hidden_size=lstm_hidden_size, num_layers =lstm_num_layers)
    
    self.cnn_1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=5, kernel_size=(5,1), stride = 1),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=5, out_channels=5, kernel_size=(5,1), stride = 1),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=5, out_channels=5, kernel_size=(5,1), stride = 1),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU()
    )

    self.cnn_2 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=5, kernel_size=(2,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=5, out_channels=5, kernel_size=(2,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=5, out_channels=5, kernel_size=(2,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU()
    )

    self.cnn_3 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=5, kernel_size=(3,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=5, out_channels=5, kernel_size=(3,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=5, out_channels=5, kernel_size=(3,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU()
    )

    self.cnn_4 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=5, kernel_size=(7,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=5, out_channels=5, kernel_size=(7,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=5, out_channels=5, kernel_size=(7,1)),
        nn.MaxPool2d(kernel_size = 4, stride = 1),
        nn.ReLU()
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(434340, 128),
        nn.ReLU(),
        nn.Dropout(0.6),
        nn.Linear(128, output_size)
    )
  def forward(self,input, h = None, c=None):
    
    embedded = self.embedding_output(input)
    lstm_input = self.process_input(embedded[2])
    
    if h == None and c==None:
      lstm_output, (h,c) = self.lstm(lstm_input) 
    else:
      lstm_output, (h,c) = self.lstm(lstm_input, (h, c))

    cnn_input = lstm_output.unsqueeze_(dim = 1)
    out_1 = self.cnn_1(cnn_input)
    print(f'out_1: {out_1.size()}')
    out_2 = self.cnn_2(cnn_input)
    print(f'out_2: {out_2.size()}')
    out_3 = self.cnn_3(cnn_input)
    print(f'out_3: {out_3.size()}')
    out_4 = self.cnn_4(cnn_input)
    print(f'out_4: {out_4.size()}')

    # Here we are going to concateate the outputs
    conc_out = torch.cat([out_1, out_2, out_3, out_4], dim =2)
    # Apply Dense and then to a FCC
    class_scores = self.classifier(conc_out)

    return class_scores
  def embedding_output(self, token):
    self.bert_embedding.eval()
    with torch.no_grad():
      output = self.bert_embedding(token)
      return output

  def process_input(self, input):
    token_embd = torch.stack(input, dim= 0)
    final_t_e = token_embd.permute(1,2,0,3)

    words_vec = []
    for seq in final_t_e:
      token_vec_cat = []
      for item in seq:
        cat_vec = torch.cat((item[-1], item[-2], item[-3], item[-4]), dim = 0)
        token_vec_cat.append(cat_vec)
      words_vec.append(torch.stack(token_vec_cat))
    return torch.stack(words_vec)

**Hyper Parameters**

In [10]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base", output_hidden_states = True)

bertweet_tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast = False, normalization = True)


Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# dataset and dataloader parameters
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
valid_df = pd.read_csv('/content/valid.csv')

max_word_count = 35

# model parameters
lstm_input_size = 3072
lstm_hidden_size = 768
lstm_num_layer = 2
cnn_kernels = [2,3,5,7]
out_channels = [5]
in_channels = 1
batch_size = 64
output_size = 7

# training parameters
model = LSTM_CNN(bertweet, 
                 lstm_input_size, 
                 lstm_hidden_size, 
                 lstm_num_layer, 
                 [], 
                 output_size).to(device)
lr = 0.001
epochs = 50
optimizer = torch.optim.Adam(model.parameters(), lr)
loss_func = nn.CrossEntropyLoss()
patience_level = 5
patience_count = 0

categories_index = {
    "Anger": 0,
    "Hate": 1,
    "Happiness" : 2,
    "Love" : 3,
    "Surprised": 4,
    "Sadness": 5,
    "Depression": 6
}
minimal_loss = np.NINF

best_model_path = "/content/gdrive/MyDrive/Deep Learning Project/model_params.pth"

In [12]:
train_ds = TweetsDatset(train_df)
test_ds = TweetsDatset(test_df)
valid_ds = TweetsDatset(valid_df)

In [13]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size= 60, shuffle =True)#, num_workers=4)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size= 60, shuffle =False)#, num_workers=4)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size= 60, shuffle =False)#, num_workers=4)

In [14]:
def train():
  train_loss_per_epoch = []
  valid_loss_per_epoch = []
  train_eval_per_epoch = []
  valid_eval_per_epoch = []

  for i in range(epochs):
    model.train()

    # Train epoch
    for idx, (data, targets) in enumerate(train_dl):
      optimizer.zero_grad()
      y_hat = model(data.to(device))
      loss = loss_func(y_hat, targets.to(device))
      loss.backward()
      optimizer.step()
      loss_list.append(loss.item())

    # Save this epoch avg loss
    train_loss_per_epoch.append(np.mean(loss_list))

    # Running validation loop
    valid_loss = validate()
    valid_loss_per_epoch.append(valid_loss)
    
    # evaluating the model
    train_eval, valid_eval = evaluate_model()
    train_eval_per_epoch.append(train_eval)
    valid_eval_per_epoch.append(valid_eval)

    # verify if we should continue using patience
    if patience_stop(valid_loss):
      break

  return train_eval_per_epoch, valid_eval_per_epoch

def validate():
  model.eval()
  loss_list = []
  for idx, (data, targets) in enumerate(valid_dl):
    y_hat = model(data.to(device))
    loss = loss_func(y_hat, targets.to(device))
    loss_list.append(loss.item())
  
  return np.mean(loss_list)


def patience_stop(loss):
  if loss > minimal_loss:
    paitence_count += 1
    if paitence_count == paitence_level:
      return True
    return False
  else:
    minimal_loss = loss
    patience_count = 0
    torch.save(model.state_dict(), best_model_path)
    print("Model saved")
    return False

def evaluate_model():
  model.eval()
  targets_list = []
  predictions_list = []
  train_data = 0
  valid_data = 0
  for i in range(2):
    dl = train_dl if i==0 else valid_dl
    for idx, (data, targets) in enumerate(dl):
      y_hat = model(data.to(device))
      predicitons = torch.argmax(y_hat, dim = 1)
      predictions_list = predicitons_list + predicitons.item()
      targets_list = targets_list + targets


    pred_array = np.array(predicitons_list)
    targets_array = np.array(targets_list)
    accuracy = accuracy_score(targets_array, pred_array)
    labels = np.array([0,1,2,3,4,5,6])
    percision = precision_score(targets_array, perd_array, labels, average='macro')
    recall = recall_score(targets_array, pred_array, labels, average='macro')
    
    if i==0:
      train_data = (accuracy, percision, recall)
    else:
      valid_data = (accuracy, percision, recall)

  return train_data, valid_data

In [15]:
train()

 data size:torch.Size([37]), tokens: ['<s>', 'Tri@@', 'logy@@', 'House', 'of', 'balloons', 'Kiss', 'land@@', 'Beauty', 'behind', 'the', 'mad@@', 'ness@@', 'Star@@', 'boy@@', 'My', 'dear', 'melanc@@', 'holy', 'After', 'hour@@', 's@@', 'THE', 'DA@@', 'WN', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 't', '</s>']
 data size:torch.Size([37]), tokens: ['<s>', '@USER', 'Yep', 'there', "'s", 'so', 'much', 'ideas', 'out', 'there', '.', 'But', 'sit', 'and', 'think', 'what', 'do', 'you', 'want', '.', '(', 'Song', 'that', 'hole', 'u', 'up', ',', 'mom@@', 'en', '...', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'HTTPURL', '</s>']
 data size:torch.Size([37]), tokens: ['<s>', '@USER', 'Blitz', ':', '...', 'The', 'fuck', '?', '"', 'He', 'goes', 'to', 'grab', 'it', 'again', 'now', 'looking', 'at', 'it', 'while', 'his@@', 'sing', 'A@@', 'bit', 'in', 'annoyed', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'imp', '</s>']
 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: ignored